In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyngrok==4.1.1
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# 2Cgo3ysthiZF4UGRmtAViko3bWr_4eC9s2WRv1kEWame8xih
!ngrok authtoken 2Cgo3ysthiZF4UGRmtAViko3bWr_4eC9s2WRv1kEWame8xih

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
def NTUB_string(user, pass_input):
    return '帳號: ' + user + '  密碼: ' + pass_input

In [8]:
"""## 1. 匯入套件"""

import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

import os
#import cv2
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

"""## 2. 建立模型"""

resnet_model = ResNet50(weights="imagenet", include_top=False, input_shape=(160, 160, 3))

model = Sequential()
model.add(resnet_model)
model.add(Dropout(0.5))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(3, activation="softmax"))
model.summary()   # 顯示模型摘要資訊

resnet_model.trainable = False

"""## 3. 匯入權重"""

model.load_weights("/content/drive/MyDrive/AI_共用/identify_food&menu/160*160_best_model.h5")

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

def pre(img):
    img_np = np.empty((1,160,160,3))
    img_np[0] = img
    result = model.predict(img_np)
    np.set_printoptions(formatter={'float': '{: 0.5f}'.format})
    print("全部預測結果：")
    print(result, end = "\n")
    maxindex = np.argmax(result, axis = 1)
    print("個別結果：")
    for index, i in enumerate(maxindex):
        print("第" + str(index + 1) + "張：", end = "")
        if i == 0:
            print("食物")
            return ("食物")
        if i == 1:
            print("非食物")
            return ("非食物")
        if i == 2:
            print("菜單")
            return ("菜單")

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 5, 5, 2048)        23587712  
                                                                 
 dropout_6 (Dropout)         (None, 5, 5, 2048)        0         
                                                                 
 global_average_pooling2d_3   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_7 (Dropout)         (None, 2048)              0         
                                                                 
 dense_3 (Dense)             (None, 3)                 6147      
                                                                 
Total params: 23,593,859
Trainable params: 23,540,739
Non-trainable params: 53,120
_____________________________________

In [10]:
from IPython.core.display import Image
from flask import Flask
from flask import request
from flask_ngrok import run_with_ngrok

from PIL import Image

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/', methods = ['GET', 'POST'])
def index():
    if request.method == 'GET':
        return """
        <form method='post' action='' enctype="multipart/form-data">
            <p><input type='file' name='image' accept= "image/gif, image/jpeg, image/jpg, image/png"></p>
            <p><button type='submit'>確定</button></p>
        </form>
        """
    else:
        try:
            img = Image.open(request.files['image'].stream).resize((160, 160)).convert('RGB')
            result = pre(img)
            
            return result
        except Exception as e:
            print("錯誤：", e)
            
            return e

if __name__ == '__main__':
    app.run() #host='0.0.0.0'

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d328-35-226-30-37.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [01/Sep/2022 07:49:38] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2022 07:49:39] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2022 07:49:46] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2022 07:50:43] "POST / HTTP/1.1" 200 -


全部預測結果：
[[ 0.00000  0.99943  0.00057]]
個別結果：
第1張：非食物


INFO:werkzeug:127.0.0.1 - - [01/Sep/2022 07:51:34] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2022 07:52:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2022 07:52:27] "POST / HTTP/1.1" 200 -


全部預測結果：
[[ 0.92564  0.07436  0.00001]]
個別結果：
第1張：食物


INFO:werkzeug:127.0.0.1 - - [01/Sep/2022 07:53:23] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2022 07:53:36] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2022 07:53:42] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2022 07:54:35] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2022 07:54:56] "POST / HTTP/1.1" 200 -


全部預測結果：
[[ 0.92564  0.07436  0.00001]]
個別結果：
第1張：食物
